In [1]:
import numpy as np
def BB(w0,wt,t,m):
    h = 2**m
    T = np.linspace(0,t, h+1)
    W = np.zeros(h+1)
    z = np.random.randn(h+1)
    W[0] = w0
    W[h] = wt
    j_max = 1
    
    for k in range(1, m+1):
        i_min = h//2
        i = i_min
        l = 0
        r = h
        for j in range(1, j_max+1):
            a = ((T[r]-T[i])*W[l] + (T[i]-T[l])*W[r]) / (T[r]-T[l])
            b = np.sqrt( (T[i]-T[l])*(T[r]-T[i])/(T[r]-T[l]) )
            W[i] = a + b*z[i]
            # print("i={}, left={}, right={}".format(i,l,r))
            i+=h; l+=h; r+=h
        j_max*=2
        h = i_min
    
    return W

W=BB(0,8,1,3)
W

array([0.        , 0.65329347, 1.89772344, 3.17295494, 4.06712841,
       4.95459829, 6.07241293, 6.5507082 , 8.        ])

In [2]:
import numpy as np
def ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,m):
    
    S1_start = 100
    S2_start = 100
    principal = 100    
    
    T = 2
    check_term = 0.5
    check_max = int(T / check_term)
    
    case=6
    S1 = np.zeros(check_max+1)
    S2 = np.zeros(check_max+1)
    payment = np.zeros(M)
    prob = np.zeros(case)
    
    for i in range(M):
        
        S1[0] = S1_start
        S2[0] = S2_start
        
        x1 = np.random.randn(check_max)
        x2 = np.random.randn(check_max)
        W1 = np.zeros(check_max+1)
        W2 = np.zeros(check_max+1)

        check_barrier = False
        
        for j in range(1, check_max+1):

            # Random walk
            W1[j] = W1[j-1] + np.sqrt(check_term) * x1[j-1]
            W2[j] = W2[j-1] + np.sqrt(check_term) * x2[j-1]
            
            e1 =  W1[j] - W1[j-1]
            e2 = rho*e1 + (W2[j] - W2[j-1])*np.sqrt(1-rho**2)
    
            S1[j] = S1[j-1]*np.exp( (r - q1 - (sigma1**2)/2)*check_term + sigma1*e1)
            S2[j] = S2[j-1]*np.exp( (r - q2 - (sigma2**2)/2)*check_term + sigma2*e2)
            
            # Check Barrier
            if (S1[j] < S1_start * 0.6) or (S2[j] < S2_start * 0.6):
                check_barrier = True
            
            # Check early repayment
            if (S1[j] >= S1_start * (0.85 - ((j-1) * 0.05))) and (S2[j] >= S2_start * (0.85 - ((j-1) * 0.05))):
                payment[i] = principal * (1+0.0625*j) * np.exp(-r * j * check_term)
                prob[j-1]+=1
                break
            
            # Check Maturity repayment
            if j == check_max:
                if (check_barrier == True) and ( (S1[j] < S1_start * 0.7) or (S2[j] < S2_start * 0.7) ):
                    payment[i] = principal * min(S1[j]/S1_start, S2[j]/S2_start) * np.exp(-r * j * check_term)
                    prob[j+1]+=1
                    break
                    
                else:
                    S1_BB = np.zeros(2**m+1)
                    S2_BB = np.zeros(2**m+1)
                    S1_BB[0] = S1_start
                    S2_BB[0] = S2_start
                    
                    # Brownian Bridge
                    for k in range(1, check_max+1):      
                        W1_BB = BB(W1[k-1], W1[k], 0.5, m)
                        W2_BB = BB(W2[k-1], W2[k], 0.5, m)
                        
                        for l in range(1, 2**m+1):
                            e1 = W1_BB[l] - W1_BB[l-1]
                            e2 = rho*e1 + (W2_BB[l] - W2_BB[l-1]) * np.sqrt(1-rho**2)
                        
                            S1_BB[l] = S1_BB[l-1] * np.exp( (r - q1 - sigma1**2/2)*(check_term/2**m) + sigma1 * e1 )
                            S2_BB[l] = S2_BB[l-1] * np.exp( (r - q2 - sigma2**2/2)*(check_term/2**m) + sigma2 * e2 )
                        
                        if any(S1_BB < S1_start * 0.6) or any(S2_BB < S2_start * 0.6):
                            check_barrier = True
                    
                        if (check_barrier == True) and ( (S1[j] < S1_start * 0.7) or (S2[j] < S2_start * 0.7) ):
                            payment[i] = principal * min(S1[j]/S1_start, S2[j]/S2_start) * np.exp(-r * j * check_term)
                            prob[j+1]+=1
                            break
                        else:        
                            S1_BB[0] = S1_BB[2**m]
                            S2_BB[0] = S2_BB[2**m]
                    
                    if (check_barrier == False) and ( (S1[j] < S1_start * 0.7) or (S2[j] < S2_start * 0.7) ):
                        payment[i] = principal * np.exp(-r * j * check_term)
                        prob[j]+=1
                        break
            
    
    value = np.mean(payment)
    err = np.std(payment) / np.sqrt(M)
    prob = prob / M
    
    return value,err,prob
                                        
from datetime import datetime
r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=10000
m=3
t1=datetime.now()
value,err,prob=ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,m)
t2=datetime.now()
print('value = {:.2f}, err = {:.2f}'.format(value,err))
for i in range(6):
    print('prob[{:d}] = {:{width}.2%}'.format(i+1,prob[i],width=6))
print('Total sum of prob = {:.0%}'.format(sum(prob)))
print('Total computing time = {:f} seconds'.format((t2-t1).total_seconds()))

value = 92.66, err = 0.27
prob[1] = 55.31%
prob[2] = 11.69%
prob[3] =  6.11%
prob[4] =  4.14%
prob[5] =  0.35%
prob[6] = 22.40%
Total sum of prob = 100%
Total computing time = 0.317160 seconds


In [3]:
import numpy as np
from datetime import datetime
r=0.03
q1=0.0
q2=0.0
sigma1=0.3
sigma2=0.4
rho=0.2
M=10000
m=7
p=np.zeros(m)
print('      ',end=' ')
for i in range(6):
    print('prob[{:d}]'.format(i+1),end=' ')
print('  computing time')
for i in range(m):
    t1=datetime.now()
    value,err,prob=ELS_BB(r,q1,q2,sigma1,sigma2,rho,M,i)
    t2=datetime.now()
    print('n={:3d}'.format(2**i),end='   ')
    for j in range(len(prob)):
        print('{:{width}.2%} '.format(prob[j],width=6),end=' ')
    print(' {:f} seconds'.format((t2-t1).total_seconds()))

       prob[1] prob[2] prob[3] prob[4] prob[5] prob[6]   computing time
n=  1   54.69%  12.16%   6.55%   4.55%   1.56%  20.49%   0.304197 seconds
n=  2   55.07%  12.63%   6.40%   4.21%   0.96%  20.73%   0.301186 seconds
n=  4   55.54%  11.93%   6.15%   4.30%   0.76%  21.32%   0.295244 seconds
n=  8   54.58%  12.75%   6.51%   4.67%   0.45%  21.04%   0.344591 seconds
n= 16   54.84%  12.53%   6.26%   4.43%   0.19%  21.75%   0.375606 seconds
n= 32   55.11%  12.60%   6.13%   4.06%   0.22%  21.88%   0.481675 seconds
n= 64   55.85%  11.87%   6.00%   4.01%   0.22%  22.05%   0.660818 seconds
